## Johdanto regressioon - Oppitunti 1

#### Aiheen hahmottaminen

✅ Regressiomenetelmiä on monenlaisia, ja valinta riippuu siitä, millaiseen kysymykseen etsit vastausta. Jos haluat ennustaa todennäköisen pituuden tietyn ikäiselle henkilölle, käyttäisit `lineaarista regressiota`, koska etsit **numeerista arvoa**. Jos taas haluat selvittää, pitäisikö tiettyä ruokakulttuuria pitää vegaanisena vai ei, etsit **luokittelua**, jolloin käyttäisit `logistista regressiota`. Opit lisää logistisesta regressiosta myöhemmin. Mieti hetki, millaisia kysymyksiä voisit esittää datalle, ja mikä näistä menetelmistä olisi sopivampi.

Tässä osiossa työskentelet [pienen diabetesdatan kanssa](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Kuvittele, että haluaisit testata hoitoa diabeetikoille. Koneoppimismallit voisivat auttaa sinua määrittämään, mitkä potilaat reagoisivat hoitoon paremmin, perustuen muuttujien yhdistelmiin. Jopa hyvin yksinkertainen regressiomalli, kun se visualisoidaan, saattaa paljastaa tietoa muuttujista, jotka auttaisivat sinua järjestämään teoreettisia kliinisiä kokeita.

Siispä, aloitetaan tämä tehtävä!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Taiteilija: @allison_horst</figcaption>

<!--![Taiteilija: \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Taiteilija: @allison_horst-->


## 1. Työkalujen lataaminen

Tätä tehtävää varten tarvitsemme seuraavat paketit:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [kokoelma R-paketteja](https://www.tidyverse.org/packages), jotka on suunniteltu tekemään datatieteestä nopeampaa, helpompaa ja hauskempaa!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) -kehys on [kokoelma paketteja](https://www.tidymodels.org/packages/) mallintamiseen ja koneoppimiseen.

Voit asentaa ne seuraavasti:

`install.packages(c("tidyverse", "tidymodels"))`

Alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa puuttuvat paketit puolestasi.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Ladataan nyt nämä mahtavat paketit ja tehdään ne saataville nykyiseen R-istuntoomme. (Tämä on vain havainnollistamista varten, `pacman::p_load()` teki sen jo puolestasi)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabetes-aineisto

Tässä harjoituksessa käytämme regressiotaitojamme ennustamalla diabetes-aineiston avulla. [Diabetes-aineisto](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) sisältää `442 näytettä` diabetekseen liittyvistä tiedoista, joissa on 10 ennustavaa muuttujaa: `ikä`, `sukupuoli`, `painoindeksi`, `keskimääräinen verenpaine` ja `kuusi veren seerumimittausta` sekä tulosmuuttuja `y`: kvantitatiivinen mittari taudin etenemisestä vuoden kuluttua lähtötilanteesta.

|Havaintojen määrä|442|
|------------------|:---|
|Ennustavien muuttujien määrä|Ensimmäiset 10 saraketta ovat numeerisia ennustavia|
|Tulos/Tavoite|Sarakkeessa 11 on kvantitatiivinen mittari taudin etenemisestä vuoden kuluttua lähtötilanteesta|
|Ennustemuuttujien tiedot|- ikä vuosina
||- sukupuoli
||- bmi painoindeksi
||- bp keskimääräinen verenpaine
||- s1 tc, kokonaisseerumin kolesteroli
||- s2 ldl, matalatiheyksiset lipoproteiinit
||- s3 hdl, korkeatiheyksiset lipoproteiinit
||- s4 tch, kokonaiskolesteroli / HDL
||- s5 ltg, mahdollisesti seerumin triglyseridien logaritmi
||- s6 glu, verensokeritaso|

> 🎓 Muista, tämä on ohjattua oppimista, ja tarvitsemme nimettyä 'y'-tavoitemuuttujaa.

Ennen kuin voit käsitellä dataa R:llä, sinun täytyy tuoda data R:n muistiin tai luoda yhteys dataan, jonka avulla R voi käyttää dataa etänä.

> [readr](https://readr.tidyverse.org/) -paketti, joka on osa Tidyversea, tarjoaa nopean ja käyttäjäystävällisen tavan lukea suorakulmaista dataa R:ään.

Ladataan nyt diabetes-aineisto, joka on saatavilla tässä lähde-URL:ssä: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Lisäksi teemme datalle tarkistuksen käyttämällä `glimpse()`-funktiota ja näytämme ensimmäiset 5 riviä `slice()`-funktiolla.

Ennen kuin jatkamme pidemmälle, esitellään myös jotain, mitä kohtaat usein R-koodissa 🥁🥁: putkioperaattori `%>%`

Putkioperaattori (`%>%`) suorittaa operaatioita loogisessa järjestyksessä siirtämällä objektin eteenpäin funktioon tai kutsulausekkeeseen. Voit ajatella putkioperaattorin tarkoittavan "ja sitten" koodissasi.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` näyttää, että tässä datassa on 442 riviä ja 11 saraketta, joista kaikki ovat tietotyyppiä `double`.

<br>

> glimpse() ja slice() ovat [`dplyr`](https://dplyr.tidyverse.org/) -kirjaston funktioita. Dplyr, joka on osa Tidyversea, on datan käsittelyn kieli, joka tarjoaa yhtenäisen joukon verbejä, jotka auttavat ratkaisemaan yleisimmät datan käsittelyn haasteet.

<br>

Nyt kun meillä on data, keskitytään yhteen ominaisuuteen (`bmi`) tämän harjoituksen kohteena. Tämä vaatii haluttujen sarakkeiden valitsemista. Miten tämä tehdään?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) mahdollistaa sarakkeiden *valitsemisen* (ja tarvittaessa uudelleennimeämisen) dataframessa.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Koulutus- ja testausdata

Ohjatussa oppimisessa on yleistä käytäntöä *jakaa* data kahteen osajoukkoon: (yleensä suurempaan) joukkoon, jolla mallia koulutetaan, ja pienempään "pidätettyyn" joukkoon, jolla nähdään, miten malli suoriutui.

Nyt kun meillä on data valmiina, voimme tarkistaa, voiko kone auttaa määrittämään loogisen jaon tämän datasetin lukujen välillä. Voimme käyttää [rsample](https://tidymodels.github.io/rsample/) -pakettia, joka on osa Tidymodels-kehystä, luodaksemme objektin, joka sisältää tiedot siitä, *miten* data jaetaan, ja sitten käyttää kahta muuta rsample-funktiota saadaksemme luodut koulutus- ja testausjoukot:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Kouluta lineaarinen regressiomalli Tidymodelsilla

Nyt olemme valmiita kouluttamaan mallimme!

Tidymodelsissa mallit määritellään käyttämällä `parsnip()`-funktiota, jossa määritellään kolme käsitettä:

-   Mallin **tyyppi** erottaa toisistaan mallit, kuten lineaarinen regressio, logistinen regressio, päätöspuumallit ja niin edelleen.

-   Mallin **tila** sisältää yleisiä vaihtoehtoja, kuten regressio ja luokittelu; jotkut mallityypit tukevat molempia, kun taas toiset toimivat vain yhdessä tilassa.

-   Mallin **moottori** on laskennallinen työkalu, jota käytetään mallin sovittamiseen. Usein nämä ovat R-paketteja, kuten **`"lm"`** tai **`"ranger"`**.

Tämä mallinnustieto tallennetaan mallin määrittelyyn, joten luodaan sellainen!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Kun malli on *määritelty*, malli voidaan `arvioida` tai `kouluttaa` käyttämällä [`fit()`](https://parsnip.tidymodels.org/reference/fit.html)-funktiota, yleensä kaavan ja datan avulla.

`y ~ .` tarkoittaa, että sovitamme `y`:n ennustettavana määränä/kohteena, jota selittävät kaikki ennustajat/piirteet eli `.` (tässä tapauksessa meillä on vain yksi ennustaja: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Mallin tuloksista voimme nähdä koulutuksen aikana opitut kertoimet. Ne edustavat parhaiten sovitetun suoran kertoimia, jotka antavat meille pienimmän kokonaisvirheen todellisen ja ennustetun muuttujan välillä.
<br>

## 5. Ennusteiden tekeminen testijoukolle

Nyt kun olemme kouluttaneet mallin, voimme käyttää sitä ennustamaan sairauden etenemistä y testidatasta käyttäen [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Tätä käytetään piirtämään viiva dataryhmien välille.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Juuri koulutimme mallin ja käytimme sitä ennusteiden tekemiseen!

Ennusteita tehdessä tidymodels-käytäntö on aina tuottaa tuloksista tibble/data frame, jossa on standardoidut sarakenimet. Tämä helpottaa alkuperäisen datan ja ennusteiden yhdistämistä käyttökelpoiseen muotoon myöhempiä toimintoja, kuten visualisointia, varten.

`dplyr::bind_cols()` yhdistää tehokkaasti useita data frameja sarakkeittain.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Mallinnustulosten visualisointi

Nyt on aika tarkastella tätä visuaalisesti 📈. Luomme hajontakaavion kaikista testijoukon `y`- ja `bmi`-arvoista ja käytämme ennusteita piirtääksemme viivan sopivimpaan kohtaan mallin dataryhmien väliin.

R:llä on useita järjestelmiä kaavioiden tekemiseen, mutta `ggplot2` on yksi tyylikkäimmistä ja monipuolisimmista. Sen avulla voit koostaa kaavioita **yhdistämällä itsenäisiä komponentteja**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Mieti hetki, mitä tässä tapahtuu. Suora viiva kulkee monien pienten datapisteiden läpi, mutta mitä se tarkalleen ottaen tekee? Voitko nähdä, miten tämän viivan avulla pitäisi pystyä ennustamaan, mihin uusi, ennennäkemätön datapiste sijoittuu suhteessa kuvaajan y-akseliin? Yritä pukea sanoiksi tämän mallin käytännön hyöty.

Onnittelut, loit ensimmäisen lineaarisen regressiomallisi, teit ennusteen sen avulla ja esitit sen kuvaajassa!



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
